<a href="https://colab.research.google.com/github/Unity-2dgame-bit/MyUnityGame/blob/main/%E5%A4%AA%E9%BC%93%E3%81%AE%E9%81%94%E4%BA%BA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# --- 1. 必要ライブラリのインストール ---
!pip install -U demucs librosa soundfile numpy

  Using cached numpy-2.4.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)


In [22]:
# --- 2. 音楽ファイルアップロード ---
from google.colab import files
uploaded = files.upload()

import os
import glob
import numpy as np
import librosa
import soundfile as sf
import zipfile

# --- 3. 対応拡張子の音楽ファイルを取得 ---
SUPPORTED_EXTS = (".mp3", ".wav", ".flac", ".ogg", ".m4a", ".aac", ".wma", ".aiff")
audio_files = [f for f in os.listdir() if f.lower().endswith(SUPPORTED_EXTS)]

if not audio_files:
    raise RuntimeError("音楽ファイルが見つかりません")


Saving デートプラン  flower × ゲキヤク.mp3 to デートプラン  flower × ゲキヤク.mp3


In [23]:
import os


In [24]:
# 先頭のファイルを使う（必要に応じて複数対応可能）
audio_file = audio_files[0]
audio_base = os.path.splitext(audio_file)[0]  # 拡張子なし
print("使用する音源:", audio_file)

# --- 4. Demucs で音源分離 ---
!pip install torchcodec

!demucs "$audio_file"

# --- 5. vocals.wav を自動探索 ---
vocals_candidates = glob.glob("separated/**/vocals.wav", recursive=True)
if not vocals_candidates:
    raise RuntimeError("vocals.wav が見つかりません（demucs失敗の可能性）")

vocals_path = vocals_candidates[0]
print("使用する vocals:", vocals_path)


使用する音源: paranoia vocalo.mp3
Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs
Separating track paranoia vocalo.mp3
100%|████████████████████████████████████████████████████████████████████████| 234.0/234.0 [00:10<00:00, 22.48seconds/s]
/usr/local/lib/python3.12/dist-packages/torchaudio/__init__.py:178: UserWarning: The 'encoding' parameter is not fully supported by TorchCodec AudioEncoder.
  return save_with_torchcodec(
/usr/local/lib/python3.12/dist-packages/torchaudio/__init__.py:178: UserWarning: The 'bits_per_sample' parameter is not directly supported by TorchCodec AudioEncoder.
  return save_with_torchcodec(
使用する vocals: separated/htdemucs/paranoia 

In [25]:
# --- 6. ボーカル読み込み & BPM 推定 ---
y, sr = librosa.load(vocals_path, sr=None)
tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
tempo = float(np.atleast_1d(tempo)[0])
print("推定 BPM:", tempo)

# --- 7. オンセット（音の立ち上がり）検出 ---
onset_frames = librosa.onset.onset_detect(y=y, sr=sr, backtrack=True, delta=0.2)
onset_times = librosa.frames_to_time(onset_frames, sr=sr)

# --- 8. 16分音符グリッドに量子化 ---
sixteenth = 60 / tempo / 4
end_time = onset_times[-1] if len(onset_times) > 0 else 0

grid_times = np.arange(0, end_time + sixteenth, sixteenth)
notes = np.zeros(len(grid_times), dtype=int)

for t in onset_times:
    idx = np.argmin(np.abs(grid_times - t))
    notes[idx] = 1  # 仮で全部ドン

# --- 9. 小節（16ノーツ）単位に分割 ---
MEASURE_LEN = 16
measures = [notes[i:i + MEASURE_LEN] for i in range(0, len(notes), MEASURE_LEN)]


推定 BPM: 74.89809782608695


In [26]:
# --- 10. tja ファイル生成 ---
tja_path = f"{audio_base}.tja"
with open(tja_path, "w", encoding="shift_jis") as f:
    f.write(f"#TITLE:{audio_base}\n")   # ← 曲名をファイル名に
    f.write(f"#BPM:{round(tempo, 2)}\n")
    f.write(f"#WAVE:{audio_file}\n")   # ← 音源ファイル名をセット
    f.write("#COURSE:Oni\n")
    f.write("#LEVEL:8\n")
    f.write("#MEASURE:4/4\n")
    f.write("#START\n")

    for m in measures:
        f.write("".join("1" if n else "0" for n in m) + ",\n")

    f.write("#END\n")

print("tja 作成完了:", tja_path)

# --- 11. 元音源を music.wav に変換（必要に応じて） ---
y_orig, sr_orig = librosa.load(audio_file, sr=None)
sf.write("music.wav", y_orig, sr_orig)


tja 作成完了: paranoia vocalo.tja


In [29]:
# --- 1. 必要ライブラリのインストール ---
!pip install -U demucs librosa soundfile numpy torchcodec

import os
import glob
import shutil
import numpy as np
import librosa
import soundfile as sf
import zipfile
from google.colab import files

# --- 2. 音楽ファイルアップロード ---
uploaded = files.upload()
SUPPORTED_EXTS = (".mp3", ".wav", ".flac", ".ogg", ".m4a", ".aac", ".wma", ".aiff")
audio_files = [f for f in os.listdir() if f.lower().endswith(SUPPORTED_EXTS)]

if not audio_files:
    raise RuntimeError("音楽ファイルが見つかりません")

audio_file = audio_files[0]
audio_base = os.path.splitext(audio_file)[0]
# 作業用フォルダ作成
export_dir = audio_base
os.makedirs(export_dir, exist_ok=True)

print(f"処理開始: {audio_file}")

# --- 4. Demucs で音源分離 ---
!demucs "$audio_file"

vocals_candidates = glob.glob("separated/**/vocals.wav", recursive=True)
if not vocals_candidates:
    raise RuntimeError("vocals.wav が見つかりません")
vocals_path = vocals_candidates[0]

# --- 6. 解析 (BPM & オフセット) ---
y, sr = librosa.load(vocals_path, sr=None)
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
tempo = float(np.atleast_1d(tempo)[0])
beat_times = librosa.frames_to_time(beat_frames, sr=sr)
offset = -beat_times[0] if len(beat_times) > 0 else 0

# --- 7. 譜面生成 (16分音符) ---
onset_frames = librosa.onset.onset_detect(y=y, sr=sr, backtrack=True, delta=0.2)
onset_times = librosa.frames_to_time(onset_frames, sr=sr)

sixteenth = 60 / tempo / 4
duration = librosa.get_duration(y=y, sr=sr)
grid_times = np.arange(0, duration, sixteenth)
notes = np.zeros(len(grid_times), dtype=int)

for t in onset_times:
    idx = np.argmin(np.abs(grid_times - t))
    notes[idx] = 1

# --- 10. tja ファイル生成 (フォルダ内に配置) ---
tja_filename = f"{audio_base}.tja"
wave_filename = f"{audio_base}.wav" # 音声ファイル名も曲名に合わせる
tja_path = os.path.join(export_dir, tja_filename)

with open(tja_path, "w", encoding="shift_jis", errors="ignore") as f:
    f.write(f"TITLE:{audio_base}\n")
    f.write(f"BPM:{round(tempo, 2)}\n")
    f.write(f"WAVE:{wave_filename}\n")
    f.write(f"OFFSET:{round(offset, 3)}\n")
    f.write("COURSE:Oni\n")
    f.write("LEVEL:8\n")
    f.write("BALLOON:\n")
    f.write("SCOREINIT:1000\n")
    f.write("SCOREDIFF:400\n")
    f.write("#START\n")

    MEASURE_LEN = 16
    for i in range(0, len(notes), MEASURE_LEN):
        measure_notes = notes[i:i + MEASURE_LEN]
        line = "".join(map(str, measure_notes))
        if len(line) < MEASURE_LEN:
            line = line.ljust(MEASURE_LEN, '0')
        f.write(line + ",\n")

    f.write("#END\n")

# --- 11. 音源をフォルダ内に保存 ---
y_orig, sr_orig = librosa.load(audio_file, sr=None)
wave_path = os.path.join(export_dir, wave_filename)
sf.write(wave_path, y_orig, sr_orig)

# --- 12. フォルダごとZIP化 ---
zip_name = f"{audio_base}_taiko_pack.zip"
with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
    for root, dirs, files_in_dir in os.walk(export_dir):
        for file in files_in_dir:
            # フォルダ構造を維持してZIPに追加
            zf.write(os.path.join(root, file),
                     os.path.relpath(os.path.join(root, file),
                     os.path.join(export_dir, '..')))

print(f"作成完了！ZIPをダウンロードします。")
files.download(zip_name)

  Using cached numpy-2.4.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)


Saving デートプラン  flower × ゲキヤク.mp3 to デートプラン  flower × ゲキヤク (2).mp3
処理開始: デートプラン  flower × ゲキヤク (1).mp3
Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs
Separating track デートプラン  flower × ゲキヤク (1).mp3
100%|██████████████████████████████████████████████| 198.89999999999998/198.89999999999998 [00:08<00:00, 22.39seconds/s]
/usr/local/lib/python3.12/dist-packages/torchaudio/__init__.py:178: UserWarning: The 'encoding' parameter is not fully supported by TorchCodec AudioEncoder.
  return save_with_torchcodec(
/usr/local/lib/python3.12/dist-packages/torchaudio/__init__.py:178: UserWarning: The 'bits_per_sample' parameter is not directly supported by TorchCodec Au

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>